In [0]:
import warnings

from keras.models import Model
from keras.layers import Input, Dense, Reshape, Lambda, dot
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence

import urllib
import urllib.request
import collections
import os
import zipfile

import numpy as np
import tensorflow as tf
print('Current TF version:', tf.__version__)

# Warning stuff 
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
    warnings.warn("user", UserWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

Using TensorFlow backend.


Current TF version: 2.2.0-rc2


In [0]:
# BRANDON'S CODE 
#Download a file if not present, and make sure it's the right size.
def maybe_download(filename, url, expected_bytes):
    if not os.path.exists(filename):
        filename, _ = urllib.request.urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if (statinfo.st_size == expected_bytes): #deleted "|| 1==1":
        print('Found and verified filename:', filename)
    else:
        print("Size: ")
        print(statinfo.st_size)
        raise Exception('Failed to verify filename:' + filename)
    return filename

#Extract the first file enclosed in a zip file as a list of words.
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

In [0]:
#Process raw inputs into a dataset.
def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

#10000
#Gets data as raw info from specified sites
def collect_data(vocabulary_size=100000):
    dataUrl = 'https://github.com/bdanver/genderBiasProject/blob/master/maleShort.zip?raw=true'
    data1 = maybe_download('maleShort.zip', dataUrl, 549335)

    vocabulary = read_data(data1)

    print(vocabulary[:20])
    data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,
                                                                vocabulary_size)
    del vocabulary  # Hint to reduce memory.
    return data, count, dictionary, reverse_dictionary

In [0]:
vocab_size = 10000
data, count, dictionary, reverse_dictionary = collect_data(vocabulary_size=vocab_size)
print(data[:7])

Found and verified filename: maleShort.zip
['www.linkedin.com/in/memler(LinkedIn)', 'Summary', 'I', 'am', 'a', 'non-traditional', 'technologist,', 'a', 'passionate', 'professional', 'with', 'experience', 'managing', 'large', 'scale', 'on-premise', 'virtualized', 'environments,', 'planning', 'and']
[210, 211, 13, 212, 5, 213, 214]


Extract all files from maleShort.zip

If you get funky error just clear output

In [0]:
from zipfile import ZipFile 

file_name = "maleShort.zip"
# Brandon deleted a def here that simply set file_name to the filename, because it 
# was never called and didn't impact the run if you call it or don't
# Cool thanks :-) -BD  

# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    
    # printing all the contents of the zip file 
    zip.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 

File Name                                             Modified             Size
maleShort/m-0001-short.txt                     2020-03-30 17:06:28         9714
maleShort/m-0002-short.txt                     2020-03-30 17:06:28         6029
maleShort/m-0003-short.txt                     2020-03-30 17:06:28         2066
maleShort/m-0004-short.txt                     2020-03-30 17:06:28        11022
maleShort/m-0005-short.txt                     2020-03-30 17:06:28         2616
maleShort/m-0006-short.txt                     2020-03-30 17:06:28         8963
maleShort/m-0007-short.txt                     2020-03-30 17:06:28         2810
maleShort/m-0008-short.txt                     2020-03-30 17:06:28         8970
maleShort/m-0009-short.txt                     2020-03-30 17:06:28         2962
maleShort/m-0010-short.txt                     2020-03-30 17:06:28         5294
maleShort/m-0011-short.txt                     2020-03-30 17:06:28         6576
maleShort/m-0012-short.txt              

Most frequent words from all of the files

stop_words = words to exclude from top words

In [0]:
from os import listdir
from os.path import isfile, join
import re
from collections import Counter

path="maleShort/"
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
  
# Removes years/numbers, single letters, ' • weird symbols, etc 
def remove_garbage(text):
    text = re.sub(r'\W+|\d+', ' ', text)
    text_final = re.sub(r'(?:^| )\w(?:$| )',' ', text)
    text_final= text_final.lower()
    return text_final

# Could import from nltk.corpus import stopwords and add to it 
stop_words = {'the','and', 'to', 'in', 'for', 'with', 'on', 'linkedin', 'of', 
              'www', 'as', 'is', 'an', 'com', 'months', 'c', 'm', 'years', 'am'}
months = {'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december'}
counter = Counter()

for file in onlyfiles:
        with open(join(path, file), 'r') as f:
          counter.update(remove_garbage(f.read()).split())

top_words = 100
for word, count in counter.most_common(top_words):
    if word not in stop_words and word not in months:
      print('{}: {}'.format(count, word))

1300: software
882: data
827: development
683: using
635: page
607: web
594: developer
580: that
579: team
566: year
554: engineer
541: experience
516: my
499: new
477: from
468: area
437: by
435: developed
428: application
393: at
376: design
363: applications
350: project
350: system
337: research
334: present
331: business
323: systems
321: code
320: management
313: education
311: was
296: worked
269: time
264: testing
261: projects
259: services
259: learning
258: summary
254: created
251: used
250: java
250: javascript
249: product
248: end
243: based
239: support
239: work
236: computer
230: have
230: including
227: full
224: engineering
223: analysis
223: through
221: all
221: greater
218: this
218: designed
217: react
211: implemented
211: working
211: it
210: stack
209: university
208: senior
205: our
203: lead
203: company
202: month
200: server
198: into
198: api
196: client
